PONTIFICA UNIVERSIDAD JAVERIANA, CALI
MAESTRÍA EN CIENCIA DE DATOS
SERIE DE TEIMPOS
UNIDAD 3: PREPROCESAMIENTO Y VISUALIZACIÓN
Estudiantes: Sandra Guzmán, Juan Sebastián y Henry Hurtado.

OBJETIVO DE LA ACTIVIDAD:
Usar gráficos y funciones de preprocesamiento de bases de datos para describir el procesamiento subyacente en la relación entre los datos en un período de tiempo.

Punto 1 - Se realiza una gráfica de serie de tiempos para la producción de café en Colombia (1990 - 2024), sin procesamiento alguno.

In [1]:
%pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 4.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import plotly.express as px

# Cargar el archivo CSV
df = pd.read_csv("FAOSTAT_Produccion_Agricola_Extendida.csv")

# Filtrar datos para Colombia y el producto Café
df_colombia_coffee = df[(df['Pais'] == 'Colombia') & (df['Producto'] == 'Coffee')]

# Ordenar por año
df_colombia_coffee = df_colombia_coffee.sort_values(by='Año')

# Crear gráfico de serie de tiempo
fig = px.line(df_colombia_coffee, x='Año', y='Produccion',
              title='Producción de Café en Colombia (1990-2024)',
              labels={'Produccion': 'Producción (toneladas)', 'Año': 'Año'})

# Mostrar el gráfico
fig.show()

# Si deseas guardar el gráfico:
fig.write_image("produccion_cafe_colombia.png")
fig.write_json("produccion_cafe_colombia.json")

FileNotFoundError: [Errno 2] No such file or directory: 'FAOSTAT_Produccion_Agricola_Extendida.csv'

Comentarios:
Se opbservan fluctuacines de picos ascendentes y descendente, no se logra observar una tendencia lineal claro.

Punto 2 - Segregación de la serie para los componentes de tendencia, estacionalidad, ciclo y residuos:

In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go

# Cargar el archivo CSV
df = pd.read_csv("FAOSTAT_Produccion_Agricola_Extendida.csv")

# Filtrar datos para Colombia y el producto Café
df_colombia_coffee = df[(df['Pais'] == 'Colombia') & (df['Producto'] == 'Coffee')]

# Ordenar por año
df_colombia_coffee = df_colombia_coffee.sort_values(by='Año')

# Establecer el índice de tiempo
df_colombia_coffee.set_index('Año', inplace=True)

# Aplicar descomposición aditiva
result = seasonal_decompose(df_colombia_coffee['Produccion'], model='additive', period=1, extrapolate_trend='freq')

# Crear gráfico con los componentes
fig = go.Figure()

fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Tendencia'))
fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Estacionalidad'))
fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residuos'))

fig.update_layout(title='Descomposición de la Producción de Café en Colombia (1990-2024)',
                  xaxis_title='Año',
                  yaxis_title='Valor',
                  legend_title='Componentes')

# Mostrar el gráfico
fig.show()

# Si deseas guardar el gráfico:
fig.write_image("descomposicion_cafe_colombia.png")
fig.write_json("descomposicion_cafe_colombia.json")

Comentarios:

La tendencia muestra la dirección general de la serie a lo largo del tiempo, (creciente, decreciente o estable), en la gráfica no se observa una tendencia clara, hay fluctuaciones entre picos altos y bajos.

La estacionalidad representa patrones que se repiten en intervalos regulares, ayuda a anticipar patrones cíclicos. En el gráfico en estudio no se logra observar un patrón de comportamiento permanente, se observan ciclos de picos altos y bajos, como si su comportamiento fuera aleatorio.

Residuos, no se evidencian valores atípicos o anomalías.


Se aplica diferenciación para lograr estacionalidad en primer orden.

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Cargar el archivo CSV
df = pd.read_csv("FAOSTAT_Produccion_Agricola_Extendida.csv")

# Filtrar datos para Colombia y el producto Café
df_colombia_coffee = df[(df['Pais'] == 'Colombia') & (df['Producto'] == 'Coffee')]

# Ordenar por año
df_colombia_coffee = df_colombia_coffee.sort_values(by='Año')

# Aplicar diferenciación no estacional (diferencia de primer orden)
df_colombia_coffee['Produccion_Diff'] = df_colombia_coffee['Produccion'].diff()

# Crear gráfico con la serie original y la diferenciada
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_colombia_coffee['Año'], y=df_colombia_coffee['Produccion'],
                         mode='lines+markers', name='Producción Original'))

fig.add_trace(go.Scatter(x=df_colombia_coffee['Año'], y=df_colombia_coffee['Produccion_Diff'],
                         mode='lines+markers', name='Diferenciación No Estacional'))

fig.update_layout(title='Producción de Café en Colombia: Serie Original vs Diferenciada',
                  xaxis_title='Año',
                  yaxis_title='Toneladas',
                  legend_title='Serie')

# Mostrar el gráfico
fig.show()

# Si deseas guardar el gráfico:
fig.write_image("produccion_cafe_colombia_diferenciada.png")
fig.write_json("produccion_cafe_colombia_diferenciada.json")

La linea azul es la serie original y la línea roja es la serie con una primera diferenciación. No se observa tendencia, ni estacionalidad.


Segunda difeenciación:


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Cargar el archivo CSV
df = pd.read_csv("FAOSTAT_Produccion_Agricola_Extendida.csv")

# Filtrar datos para Colombia y el producto Café
df_colombia_coffee = df[(df['Pais'] == 'Colombia') & (df['Producto'] == 'Coffee')]

# Ordenar por año
df_colombia_coffee = df_colombia_coffee.sort_values(by='Año')

# Primera diferenciación (diferencia de primer orden)
df_colombia_coffee['Diff_1'] = df_colombia_coffee['Produccion'].diff()

# Segunda diferenciación (diferencia de segundo orden)
df_colombia_coffee['Diff_2'] = df_colombia_coffee['Diff_1'].diff()

# Crear gráfico con las tres series
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_colombia_coffee['Año'], y=df_colombia_coffee['Produccion'],
                         mode='lines+markers', name='Producción Original'))

fig.add_trace(go.Scatter(x=df_colombia_coffee['Año'], y=df_colombia_coffee['Diff_1'],
                         mode='lines+markers', name='1ª Diferenciación'))

fig.add_trace(go.Scatter(x=df_colombia_coffee['Año'], y=df_colombia_coffee['Diff_2'],
                         mode='lines+markers', name='2ª Diferenciación'))

fig.update_layout(title='Producción de Café en Colombia: Original vs Diferenciaciones',
                  xaxis_title='Año',
                  yaxis_title='Toneladas',
                  legend_title='Serie')

# Mostrar el gráfico
fig.show()

# Si deseas guardar el gráfico:
fig.write_image("produccion_cafe_colombia_doble_diferenciacion.png")
fig.write_json("produccion_cafe_colombia_doble_diferenciacion.json")

Se aplica la prueba ADF sobre la serie difereniada.


In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Cargar el archivo CSV
df = pd.read_csv("FAOSTAT_Produccion_Agricola_Extendida.csv")

# Filtrar datos para Colombia y el producto Café
df_colombia_coffee = df[(df['Pais'] == 'Colombia') & (df['Producto'] == 'Coffee')]

# Ordenar por año
df_colombia_coffee = df_colombia_coffee.sort_values(by='Año')

# Primera diferenciación
df_colombia_coffee['Diff_1'] = df_colombia_coffee['Produccion'].diff()

# Segunda diferenciación
df_colombia_coffee['Diff_2'] = df_colombia_coffee['Diff_1'].diff()

# Eliminar valores NaN
serie_diff2 = df_colombia_coffee['Diff_2'].dropna()

# Aplicar prueba ADF
adf_result = adfuller(serie_diff2)

# Mostrar resultados
print("Estadístico ADF:", adf_result[0])
print("p-valor:", adf_result[1])
print("Número de rezagos:", adf_result[2])
print("Número de observaciones:", adf_result[3])
print("Valores críticos:")
for key, value in adf_result[4].items():
    print(f"   {key}: {value}")

# Interpretación:
# Si el p-valor < 0.05, la serie es estacionaria.